In [10]:
from flask import Flask, jsonify
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
old_df = pd.read_csv('ratings_Beauty.csv')


In [12]:
selected_columns = ['user_id', 'product_id', 'rating']
df = old_df[selected_columns]
df = df.drop_duplicates(['user_id', 'product_id', 'rating'])
df['rating'] = df['rating'].astype(float)  # Convert rating column to float
df = df.head(5000)


df


,user_id,product_id,rating
0,A39HTATAQ9V7YF,205616461,5.0
1,A3JM6GV9MNOF9X,558925278,3.0
2,A1Z513UWSAAO0F,558925278,5.0
3,A1WMRR494NWEWV,733001998,4.0
4,A3IAAVS479H7M7,737104473,1.0
...,...,...,...
4995,A2SPUNKFQGPH9V,B000052XZP,5.0
4996,A24X8P2TG2AR02,B000052XZP,4.0
4997,A33G7JYH0ZT2I2,B000052XZP,4.0
4998,AL6DGYW0UPICX,B000052XZP,5.0


In [13]:
# Reset user_id and product_id to sequential numbers starting from 1
df['user_id'] = df['user_id'].astype('category').cat.codes + 1
df['product_id'] = df['product_id'].astype('category').cat.codes + 1
df = df.sort_values('user_id')
df_sort_product=df.sort_values('product_id')
df

,user_id,product_id,rating
350,1,116,5.0
296,2,99,5.0
3312,3,706,3.0
3263,4,706,5.0
4663,5,744,4.0
...,...,...,...
2352,4914,673,1.0
46,4915,18,5.0
2378,4916,678,5.0
764,4917,201,4.0


In [14]:
df_sort_product

,user_id,product_id,rating
6,414,1,5.0
7,1414,2,5.0
8,3715,3,5.0
9,2668,4,4.0
10,414,4,5.0
...,...,...,...
4976,80,755,5.0
4946,2210,755,5.0
4941,389,755,4.0
4979,3363,755,3.0


In [15]:
user_id = 2786
pivot_df = df.pivot(index='user_id', columns='product_id', values='rating')
# Reset the column names
pivot_df.columns.name = None
# Reset the index name
pivot_df.index.name = None
# Calculate the average rating for each product
pivot_df

,1,2,3,4,5,6,7,8,9,10,...,746,747,748,749,750,751,752,753,754,755
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
product_avg = pivot_df.mean(axis=0)

product_avg

1      5.000000
2      5.000000
3      5.000000
4      4.333333
5      5.000000
         ...   
751    5.000000
752    4.378378
753    3.000000
754    5.000000
755    3.955224
Length: 755, dtype: float64

In [17]:
# Fill NaN values with 0
pivot_df = pivot_df.fillna(0)
# Calculate the user-product matrix by subtracting the product average from each rating
matrix_avg = pivot_df.sub(product_avg, axis=1)

matrix_avg

,1,2,3,4,5,6,7,8,9,10,...,746,747,748,749,750,751,752,753,754,755
1,-5.0,-5.0,-5.0,-4.333333,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,...,-4.0,-4.25,-4.245399,-1.0,-1.16,-5.0,-4.378378,-3.0,-5.0,-3.955224
2,-5.0,-5.0,-5.0,-4.333333,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,...,-4.0,-4.25,-4.245399,-1.0,-1.16,-5.0,-4.378378,-3.0,-5.0,-3.955224
3,-5.0,-5.0,-5.0,-4.333333,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,...,-4.0,-4.25,-4.245399,-1.0,-1.16,-5.0,-4.378378,-3.0,-5.0,-3.955224
4,-5.0,-5.0,-5.0,-4.333333,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,...,-4.0,-4.25,-4.245399,-1.0,-1.16,-5.0,-4.378378,-3.0,-5.0,-3.955224
5,-5.0,-5.0,-5.0,-4.333333,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,...,-4.0,-4.25,-4.245399,-1.0,-1.16,-5.0,-4.378378,-3.0,-5.0,-3.955224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4914,-5.0,-5.0,-5.0,-4.333333,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,...,-4.0,-4.25,-4.245399,-1.0,-1.16,-5.0,-4.378378,-3.0,-5.0,-3.955224
4915,-5.0,-5.0,-5.0,-4.333333,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,...,-4.0,-4.25,-4.245399,-1.0,-1.16,-5.0,-4.378378,-3.0,-5.0,-3.955224
4916,-5.0,-5.0,-5.0,-4.333333,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,...,-4.0,-4.25,-4.245399,-1.0,-1.16,-5.0,-4.378378,-3.0,-5.0,-3.955224
4917,-5.0,-5.0,-5.0,-4.333333,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,...,-4.0,-4.25,-4.245399,-1.0,-1.16,-5.0,-4.378378,-3.0,-5.0,-3.955224


In [18]:
# Calculate the cosine similarity matrix
similarity = cosine_similarity(matrix_avg)
# Get the row of the similarity matrix corresponding to the user
similarity

array([[1.        , 0.99822489, 0.99879269, ..., 0.99822435, 0.99854408,
        0.99822446],
       [0.99822489, 1.        , 0.99879258, ..., 0.99822419, 0.99854395,
        0.99822431],
       [0.99879269, 0.99879258, 1.        , ..., 0.99879213, 0.99911193,
        0.99985791],
       ...,
       [0.99822435, 0.99822419, 0.99879213, ..., 1.        , 0.9985434 ,
        0.99822364],
       [0.99854408, 0.99854395, 0.99911193, ..., 0.9985434 , 1.        ,
        0.99854352],
       [0.99822446, 0.99822431, 0.99985791, ..., 0.99822364, 0.99854352,
        1.        ]])

In [19]:
similarity.shape


(4918, 4918)

In [20]:
user_similarity = similarity[user_id-1]
user_similarity

array([0.99854438, 0.99854426, 0.99911217, ..., 0.9985438 , 0.99886355,
       0.9985439 ])

In [21]:
similar_users = user_similarity.argsort()[:-2-1:-1]
test_similar_users =  np.sort(user_similarity)
test_similar_users



array([0.98782381, 0.99140693, 0.99231146, ..., 0.99968051, 0.9999645 ,
       1.        ])

In [22]:
test_similar_users_arg=user_similarity.argsort()
test_similar_users_arg

array([ 413, 1413, 1968, ...,  913, 2608, 2785], dtype=int64)

In [23]:
user_similarity[89]

0.9993966536082

In [24]:
similar_users

array([2785, 2608], dtype=int64)

In [25]:
unrated_products = pivot_df.loc[user_id][pivot_df.loc[user_id] == 0].index
unrated_products


Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            746, 747, 748, 749, 750, 751, 752, 753, 754, 755],
           dtype='int64', length=754)

In [28]:
product_predictions = {}
for product in unrated_products:
    product_ratings = matrix_avg[product].values
    # Only consider ratings from similar users
    similar_ratings = product_ratings[similar_users]
    # Ignore ratings of 0 (unrated products)
    similar_ratings = similar_ratings[similar_ratings != 0]
    if len(similar_ratings) > 0:
        # Calculate the predicted rating as the weighted average of similar ratings
        weights = user_similarity[similar_users][similar_ratings.nonzero()]
        prediction = (similar_ratings * weights).sum() / weights.sum()
        product_predictions[product] = prediction
recommended_products = sorted(product_predictions, key=product_predictions.get, reverse=True)[:4]

    

In [29]:
recommended_products

[13, 35, 36, 45]